# HateSpeech Detection in hindi

# Imports

In [71]:
import pandas as pd
import numpy as np
import re
import emoji
import string
import csv
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# DataSet

In [2]:
data = pd.read_csv(r'G:/PPSU Academic/SEM  4/SSIT2930 Minor Project-II/DataSet.csv')
data.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


In [3]:
data["Post"][0]

'मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏'

# Text Cleanning - Data Pre-Processing

In [4]:
STOPWORDS = ["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप",
             "इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी",
             "इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके",
             "उनको","उन्हीं","उन्हें","उन्हों","उस","उसके",'थी,',"उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर",
             "और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा",'कहाँ',"का","काफि","काफ़ी",
             "कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर",'हम',"किस","किसि","किसी","किसे","की",
             "कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ",
             "जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे",
             "जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि",
             "थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे",
             "निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला",
             "बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि",'सैन्\u200dय',
             'सम्\u200dमान', 'नई', 'दिल्\u200dली', 'संस्\u200dकार।',
             "यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग",
             "वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि",'श्री','स्\u200dटीफन','माध्\u200dयम', 
             "सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है",
             "हैं","हो","होता","होति","होती","होते","होना","होने"'थी,', 'वो','आ','हैं.', 'नए', 'हैं,','जाएगी', 'है।','है', 'क्यों','है,']


In [5]:
def stopword_removal(new_post):
    tokens = word_tokenize(new_post)
    stopword_remove_row = [word for word in tokens if word not in STOPWORDS]
    return stopword_remove_row

In [6]:
def emojis_removal(post):
    list1 = []
    for i in post:
        allchars = [str for str in i]
        emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
        clean_text = ' '.join([str for str in i.split() if not any(i in str for i in emoji_list)])
        list1.append(clean_text)
    return list1

In [7]:
def puncuation_removal(list):
    list2 = []
    for i in list:
        new_str = re.sub(r'[a-zA-Z0-9\n\',.#@_:…।?/|!$*-]', r'',i)
        list2.append(new_str)
    return list2

In [8]:
post = data["Post"]
new_post = emojis_removal(post)
post1 = puncuation_removal(new_post)


In [9]:
post1

['मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम',
 'सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नईनई स्कीमें लाती रहती है ताकि उन पर ज्यादा आर्थिक बोझ न पड़े ',
 'सुशांत ने जो बिजनेस डील  जून को की थी वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई देखिए पूछता है भारत अर्नब के साथ रिपब्लिक भारत पर    ',
 ' साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है',
 '  अनलॉक के लिए गाइडलाइन्स जारी   सितंबर से देशभर में मेट्रो सेवा शुरु होगी   सितंबर के बाद रैलियों और बाकी फंक्शन में  लोगों को इजाजत  कंटेनमेंट जोन में कोई छूट नहीं  सिनेमाहॉल अभी बंद रहेंगे   से वीं के छात्र  सितंबर के बाद स्कूल जा सकेंगे ',
 'चीन ने  में तर्क दिया की भारत का विपक्ष ही अजर\u200c मसुद को आतंकी नहीं मानता तो हम कैसे माने चुल्लू भर मूत्र में डूब मरो गद्दारों अब यह भारत के लोगों को सोचना है कि वो विपक्ष को वोट क्यों करेंविक्रम शर्मा',
 'देश में  के रिकॉर्ड मामले   ',
 '   से निक

In [10]:
data_x = post1
data_y = data["Labels Set"]


In [11]:
len(data_x)

5728

In [12]:
len(data_y)

5728

In [13]:
data_y.head()

0          hate,offensive
1             non-hostile
2             non-hostile
3    defamation,offensive
4             non-hostile
Name: Labels Set, dtype: object

In [14]:
data_dict = { 
    
    "post": data_x,
    "labels": data_y,   
}

In [15]:
data_new = pd.DataFrame(data_dict)
data_new.head()

,post,labels
0,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के...,"hate,offensive"
1,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,सुशांत ने जो बिजनेस डील जून को की थी वो डील द...,non-hostile
3,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive"
4,अनलॉक के लिए गाइडलाइन्स जारी सितंबर से देश...,non-hostile


In [16]:
l = []
for i in data_y:
    a = i.split(",")
    for k in a:
        l.append(k)

        
print("hate", l.count("hate"))
print("offensive", l.count("offensive"))
print("non-hostile", l.count("non-hostile"))
print("defamation", l.count("defamation"))
print("fake", l.count("fake"))

hate 792
offensive 742
non-hostile 3050
defamation 564
fake 1144


In [17]:
df_dict = {
    "post":data_x,
    "hate":np.zeros(5728),
    "offensive":np.zeros(5728),
    "non-hostile":np.zeros(5728),
    "defamation":np.zeros(5728),
    "fake":np.zeros(5728)
}

In [18]:
df_new = pd.DataFrame(df_dict)

In [19]:
for i in range(0,len(data_new["labels"])):
    a = data_new["labels"][i].split(",")
    for k in a:
        df_new[k][i] = 1
        

C:\Users\Dhruv.DHRUVPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [20]:
df_new.head()

,post,hate,offensive,non-hostile,defamation,fake
0,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के...,1.0,1.0,0.0,0.0,0.0
1,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,0.0,0.0,1.0,0.0,0.0
2,सुशांत ने जो बिजनेस डील जून को की थी वो डील द...,0.0,0.0,1.0,0.0,0.0
3,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,0.0,1.0,0.0,1.0,0.0
4,अनलॉक के लिए गाइडलाइन्स जारी सितंबर से देश...,0.0,0.0,1.0,0.0,0.0


In [21]:
X = df_new["post"]
ini_array1 = np.array(df_new[df_new.columns[1:]])

# TF CountVectorizer

In [22]:
cv = CountVectorizer(analyzer=stopword_removal,)

tfvetorizar  = cv.fit_transform(X)
print(tfvetorizar.shape)


(5728, 16397)


In [23]:
print(cv.get_feature_names())

['%', '&', "''", '(', ')', '+', ';', '=', '=====', '=चीरहरण', '>', '[', '\\', ']', '``', '~', '۔۔ठेके', 'अंक', 'अंकल', 'अंकाउंट', 'अंकित', 'अंकिता', 'अंखी', 'अंग', 'अंगतस्करीव्यापार', 'अंगना', 'अंगप्रदर्शन', 'अंगारों', 'अंगूठा', 'अंगूठे', 'अंगों', 'अंग्रेज', 'अंग्रेजी', 'अंग्रेजो', 'अंग्रेजों', 'अंजना', 'अंजाम', 'अंडभक्तों', 'अंडमान', 'अंडमानवासियों', 'अंडरटेकिंग', 'अंडरवर्ल्ड', 'अंड़भक्त', 'अंडे', 'अंत', 'अंतत', 'अंततः', 'अंतर', 'अंतरंग', 'अंतरण', 'अंतरराष्ट्रीय', 'अंतरराष्\u200dट्रीय', 'अंतरसरकारी', 'अंतरात्मा', 'अंतराल', 'अंतरिक्ष', 'अंतरित', 'अंतरिम', 'अंतर्गत', 'अंतर्राष्ट्रीय', 'अंतर्राष्\u200dट्रीय', 'अंतिम', 'अंतू', 'अंतूकापूरा', 'अंत्योदय', 'अंदरखाने', 'अंदरूनी', 'अंदाज', 'अंदाज़ा', 'अंदाजा', 'अंदेशा', 'अंध', 'अंधकार', 'अंधभक्त', 'अंधभक्तन', 'अंधभक्तनी', 'अंधभक्ति', 'अंधभक्तो', 'अंधभक्तों', 'अंधविश्वास', 'अंधविस्वास', 'अंधा', 'अंधेर', 'अंधेरे', 'अंबाजी', 'अंबानी', 'अंबाला', 'अंबेडकर', 'अंश', 'अंशुमान', 'अंसल', 'अंसारी', 'अकड़', 'अकबर', 'अकल', 'अकवाम', 'अकाउंट', 'अकादमिक', 'अकि

In [24]:
df_tfvetorizar= pd.DataFrame(tfvetorizar.toarray(), columns = cv.get_feature_names())
df_tfvetorizar.head()

,%,&,'',(,),+,;,=,=====,=चीरहरण,...,•असीम,•आसिया,•खादीरुन,•मकबूल,•मुनावर,•शगुफ्ता,₹,★,ー,￼लोगों
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# TF_IDF countvectorization

In [25]:
tfidf = TfidfVectorizer(analyzer=stopword_removal)

tfidfvetorizar = tfidf.fit_transform(df_new["post"])
print(tfidfvetorizar.shape)

(5728, 16397)


In [26]:
print(tfidf.get_feature_names())

['%', '&', "''", '(', ')', '+', ';', '=', '=====', '=चीरहरण', '>', '[', '\\', ']', '``', '~', '۔۔ठेके', 'अंक', 'अंकल', 'अंकाउंट', 'अंकित', 'अंकिता', 'अंखी', 'अंग', 'अंगतस्करीव्यापार', 'अंगना', 'अंगप्रदर्शन', 'अंगारों', 'अंगूठा', 'अंगूठे', 'अंगों', 'अंग्रेज', 'अंग्रेजी', 'अंग्रेजो', 'अंग्रेजों', 'अंजना', 'अंजाम', 'अंडभक्तों', 'अंडमान', 'अंडमानवासियों', 'अंडरटेकिंग', 'अंडरवर्ल्ड', 'अंड़भक्त', 'अंडे', 'अंत', 'अंतत', 'अंततः', 'अंतर', 'अंतरंग', 'अंतरण', 'अंतरराष्ट्रीय', 'अंतरराष्\u200dट्रीय', 'अंतरसरकारी', 'अंतरात्मा', 'अंतराल', 'अंतरिक्ष', 'अंतरित', 'अंतरिम', 'अंतर्गत', 'अंतर्राष्ट्रीय', 'अंतर्राष्\u200dट्रीय', 'अंतिम', 'अंतू', 'अंतूकापूरा', 'अंत्योदय', 'अंदरखाने', 'अंदरूनी', 'अंदाज', 'अंदाज़ा', 'अंदाजा', 'अंदेशा', 'अंध', 'अंधकार', 'अंधभक्त', 'अंधभक्तन', 'अंधभक्तनी', 'अंधभक्ति', 'अंधभक्तो', 'अंधभक्तों', 'अंधविश्वास', 'अंधविस्वास', 'अंधा', 'अंधेर', 'अंधेरे', 'अंबाजी', 'अंबानी', 'अंबाला', 'अंबेडकर', 'अंश', 'अंशुमान', 'अंसल', 'अंसारी', 'अकड़', 'अकबर', 'अकल', 'अकवाम', 'अकाउंट', 'अकादमिक', 'अकि

In [28]:
df_tfidfvetorizar = pd.DataFrame(tfidfvetorizar.toarray(), columns = tfidf.get_feature_names())
df_tfidfvetorizar.head()

,%,&,'',(,),+,;,=,=====,=चीरहरण,...,•असीम,•आसिया,•खादीरुन,•मकबूल,•मुनावर,•शगुफ्ता,₹,★,ー,￼लोगों
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Split DataSet into train and test

In [29]:
X_train,X_test,Y_train,Y_test = train_test_split(df_tfidfvetorizar,ini_array1,test_size=0.33, random_state=88)

In [30]:
len(X_train)

3837

In [31]:
len(X_test)

1891

In [32]:
len(Y_train)

3837

In [33]:
len(Y_test)

1891

In [34]:
Y = ini_array1.ravel()
target_data=Y[0:len(Y_train)]

# Modal traning using Gaussian Naive Bayes

In [35]:
model = GaussianNB()
model.fit(X_train,target_data)

GaussianNB()

In [36]:
expected= target_data
predicated =model.predict(X_train)

In [37]:
print(metrics.classification_report(expected,predicated))


              precision    recall  f1-score   support

         0.0       1.00      0.91      0.96      3002
         1.0       0.76      1.00      0.87       835

    accuracy                           0.93      3837
   macro avg       0.88      0.96      0.91      3837
weighted avg       0.95      0.93      0.94      3837



In [123]:
print(metrics.confusion_matrix(expected,predicated))

[[2745  257]
 [   0  835]]


In [59]:
result=model.predict(X_test)
    

# Logistic regression

In [64]:
model = LogisticRegression()

In [65]:
model.fit(X_train,target_data)

LogisticRegression()

In [66]:
expected= target_data
predicated =model.predict(X_train)
print(metrics.classification_report(expected,predicated))

              precision    recall  f1-score   support

         0.0       0.78      1.00      0.88      3002
         1.0       0.00      0.00      0.00       835

    accuracy                           0.78      3837
   macro avg       0.39      0.50      0.44      3837
weighted avg       0.61      0.78      0.69      3837



C:\Users\Dhruv.DHRUVPC\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dhruv.DHRUVPC\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dhruv.DHRUVPC\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# KNN

In [72]:

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [78]:
knn_classifier = KNeighborsClassifier(n_neighbors = 6)
knn_classifier.fit(X_train,target_data)

KNeighborsClassifier(n_neighbors=6)

In [79]:
expected= target_data
predicated =model.predict(X_train)
print(metrics.classification_report(expected,predicated))

              precision    recall  f1-score   support

         0.0       0.78      1.00      0.88      3002
         1.0       0.00      0.00      0.00       835

    accuracy                           0.78      3837
   macro avg       0.39      0.50      0.44      3837
weighted avg       0.61      0.78      0.69      3837



C:\Users\Dhruv.DHRUVPC\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dhruv.DHRUVPC\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dhruv.DHRUVPC\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
